In [1]:
# univariate multi-step vector-output stacked LSTM example

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
#split a univariate sequence into samples

def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

In [6]:
#  define input sequence

row_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [7]:
# choose a number of time steps

n_steps_in,  n_steps_out = 3, 2

In [8]:
# split into samples

X, y = split_sequence(row_seq, n_steps_in, n_steps_out)

In [9]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70]])

In [10]:
y

array([[40, 50],
       [50, 60],
       [60, 70],
       [70, 80],
       [80, 90]])

In [11]:
# reshape from [samples, timesteps] into [samples, timesteps, features]

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [12]:
X

array([[[10],
        [20],
        [30]],

       [[20],
        [30],
        [40]],

       [[30],
        [40],
        [50]],

       [[40],
        [50],
        [60]],

       [[50],
        [60],
        [70]]])

In [13]:
# define model

model = Sequential()
model.add(LSTM(100, activation = 'relu', return_sequences = True, input_shape = (n_steps_in, n_features)))
model.add(LSTM(100, activation = 'relu'))
model.add(Dense(n_steps_out))

In [14]:
# compile the model
model.compile(optimizer = 'Rmsprop', loss = 'mse')

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 100)            40800     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 121,402
Trainable params: 121,402
Non-trainable params: 0
_________________________________________________________________


In [16]:
#fit the model
model.fit(X, y, epochs = 50, verbose = 0)

In [18]:
# demonstrate predictions

x_input = array([70, 80, 90])
x_input = x_input.reshape(1, n_steps_in, n_features)
y_pred = model.predict(x_input, verbose = 0)

In [19]:
x_input

array([[[70],
        [80],
        [90]]])

In [20]:
y_pred

array([[90.39976, 97.88961]], dtype=float32)